In [1]:
import boto3

In [2]:
region_name = 'us-east-1'

# ECS 클라이언트 생성
ecs_client = boto3.client('ecs', region_name=region_name)

In [6]:
def run_task(cluster, task_definition, file_hashkey, experiment_name, user_name, task_token, cpu=2048, memory=4096):
    cpu = str(cpu)
    memory = str(memory)
    task_role_arn = 'arn:aws:iam::891377179922:role/ecsTaskExecutionRole'
    container_name = task_definition

    # 작업 실행
    response = ecs_client.run_task(
        cluster = cluster,
        taskDefinition = task_definition,
        launchType='FARGATE',
        networkConfiguration={
            'awsvpcConfiguration': {
                'subnets': [
                    'subnet-039ccc0392e787a99',  # 실제 서브넷 ID로 대체
                ],
                'securityGroups': [
                    'sg-0b8affc31e27faf78',  # 실제 보안 그룹 ID로 대체
                ],
                'assignPublicIp': 'ENABLED'
            }
        },
        overrides={
            'taskRoleArn': task_role_arn,
            'containerOverrides': [
                {
                    'name': container_name,  # 실제 컨테이너 이름으로 대체
                    'command': [
                        'python', 'main.py',
                        '--file_hashkey', file_hashkey,
                        '--experiment_name', experiment_name,
                        '--user_name', user_name,
                        '--task_token', task_token
                    ]
                }
            ],
            'cpu': cpu,  # CPU 유닛 (예: 512 = 0.5 vCPU)
            'memory': memory  # 메모리 (MB 단위, 예: 1024 = 1GB)
        }
    )
    return response

In [7]:
# input

cluster = 'algolab-fargate-cluster'
task_definition = 'algolab-fargate-dowhy'
file_hashkey = 'morris_sample'
experiment_name = '99 trial'
user_name = 'hjj0204@gsenergy.co.kr'
task_token = '123'

cpu = 1024*2
memory = 1024*8

In [8]:
response = run_task(cluster, task_definition, file_hashkey, experiment_name, user_name, task_token, cpu, memory)

# 실행 결과 출력
print(response)

{'tasks': [{'attachments': [{'id': '79b7371c-c5a0-4888-8673-dda87ab5293e', 'type': 'ElasticNetworkInterface', 'status': 'PRECREATED', 'details': [{'name': 'subnetId', 'value': 'subnet-039ccc0392e787a99'}]}], 'attributes': [{'name': 'ecs.cpu-architecture', 'value': 'x86_64'}], 'availabilityZone': 'us-east-1c', 'clusterArn': 'arn:aws:ecs:us-east-1:891377179922:cluster/algolab-fargate-cluster', 'containers': [{'containerArn': 'arn:aws:ecs:us-east-1:891377179922:container/algolab-fargate-cluster/2e3eca0048a74425af1532f7e8846452/48753097-eb4d-46bd-b565-d2559cf3e91a', 'taskArn': 'arn:aws:ecs:us-east-1:891377179922:task/algolab-fargate-cluster/2e3eca0048a74425af1532f7e8846452', 'name': 'algolab-fargate-dowhy', 'image': '891377179922.dkr.ecr.us-east-1.amazonaws.com/algolab-fargate-dowhy:1.0', 'lastStatus': 'PENDING', 'networkInterfaces': [], 'cpu': '0'}], 'cpu': '2048', 'createdAt': datetime.datetime(2024, 8, 2, 0, 34, 7, 57000, tzinfo=tzlocal()), 'desiredStatus': 'RUNNING', 'enableExecuteComm